# Task 1c - Association Analysis

# Import

In [1]:
import pandas as pd
import numpy as np
import json

from collections import Counter
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_json('Sports_and_Outdoors_5.json',lines = True)
df_meta = pd.read_json('meta_Sports_and_Outdoors.json', lines=True)

# Übersicht der Daten

In [3]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,vote,image
0,5,True,"06 3, 2015",A180LQZBUWVOLF,0000032034,Michelle A,What a spectacular tutu! Very slimming.,Five Stars,1433289600,NaN,NaN,NaN
1,1,True,"04 1, 2015",ATMFGKU5SVEYY,0000032034,Crystal R,What the heck? Is this a tutu for nuns? I know...,Is this a tutu for nuns?!,1427846400,NaN,NaN,NaN
2,5,True,"01 13, 2015",A1QE70QBJ8U6ZG,0000032034,darla Landreth,Exactly what we were looking for!,Five Stars,1421107200,NaN,NaN,NaN
3,5,True,"12 23, 2014",A22CP6Z73MZTYU,0000032034,L. Huynh,I used this skirt for a Halloween costume and ...,I liked that the elastic waist didn't dig in (...,1419292800,NaN,NaN,NaN
4,4,True,"12 15, 2014",A22L28G8NRNLLN,0000032034,McKenna,This is thick enough that you can't see throug...,This is thick enough that you can't see throug...,1418601600,NaN,NaN,NaN


In [4]:
df.shape

(2839940, 12)

In [5]:
df_meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Adult Tutu Assorted Colors (Turquoise),[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (","[B071LQWQBQ, B00M14DG0O, B076GTTY9W, B07BSZ4GY...","<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.80,0000032042,[],[],NaN
1,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Bububibi Adult Ballet Tutu Cheetah Pink,[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (",[],"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.97,0000032069,[],[],NaN
2,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Girls Ballet Tutu Neon Orange,[B01MYHD3LV],,Unknown,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#924,198 in Toys & Games (See Top 100 in Toy...","[B0152HBC52, B07C29WWV1, B01M0BIP9H, B00VXJHFR...",Toys & Games,,,$5.70,0000031860,[],[],NaN
3,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,0000031852,[],[],NaN
4,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[Dance tutu for girls ages 2-8 years. Perfect ...,,Girls Ballet Tutu Neon Blue,"[B009THXHPA, B00BEU1ZBI, B079PYLZTM, B00BEU1ZR...",,Bububibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#393,597 in Toys & Games (See Top 100 in Toy...","[B009THXHPA, B079PYLZTM, B079PQPLT3, B00UP6A3R...",Toys & Games,,,$6.54,0000031895,[],[],NaN


In [6]:
df_meta.shape

(962300, 19)

In [ ]:
# Alle Transaktionen von allen Kunden in csv abspeichern

# g = df.groupby('reviewerID')

# t = []
# for elem in g.groups.values():
#     liste = []
#     for ele in elem:
#         liste.append(ele)
#     t.append(liste)

# k = []
# for elem in g.groups.keys():
#     k.append(elem)

# dat = {'reviewerID':k, 'transactions':t}
# dff = pd.DataFrame(data = dat)

# dff.to_csv('transactions.csv', index=False)

# Assoziations Analyse


## Allgemeine Reduzierung des Datensatzes

In [7]:
print(df['verified'].value_counts())

True     2611805
False     228135
Name: verified, dtype: int64


In [8]:
df_assoziation = df[(df['verified'] == True)]
print('Anzahl an Zeilen: ', df_assoziation.index.size)

Anzahl an Zeilen:  2611805


Alle nicht verifizierten Einkäufe werden entfernt, weil es hier keine Bestätigung des tatsächlichen Einkaufs gibt.\
Damit fallen etwa **8% der Daten weg**.

## 1. Ansatz: Einkäufe von allen Kunden über eine Zeitscheibe

### Einschränken des Zeitraums - 1.1.2015 - 30.6.2015

In [ ]:
# nur Einträge von 2015 -> mehrere TB während des OHE
# df_assoziation_1 = df_assoziation_1[(df_assoziation_1['unixReviewTime'] >= 1420066800) & (df_assoziation_1['unixReviewTime'] <= 1451602799)]
# df_assoziation_1.shape

In [9]:
# nur Einträge von 1.1.2015 - 30.6.2015
df_assoziation_1 = df_assoziation[(df_assoziation['unixReviewTime'] >= 1420066800) & (df_assoziation['unixReviewTime'] <= 1435701599)]
print('Anzahl an Zeilen: ',df_assoziation_1.index.size)

Anzahl an Zeilen:  278183


In [ ]:
# POC für Daten von 1.1.2015 - 30.4.2015
# df_assoziation_1 = df_assoziation[(df_assoziation['unixReviewTime'] >= 1420066800) & (df_assoziation['unixReviewTime'] <= 1433109599)]
# df_assoziation_1.shape

In [10]:
df_assoziation_red_1 = df_assoziation_1[['reviewerID', 'asin']]

Für die Analyse werden nur die Spalten reviewerID und asin (Code des Artikels) benötigt. Alle anderen Spalten werden verworfen um die Performance später zu verbessern.

### Gruppierung der Einträge mit For-Schleife

In [ ]:
# customers = df2['reviewerID']
# transactions = []

# Alle Kunden (332447) -> nach 3 Stunden kein Ergebnis
# for x in customers:
#     ser = df.loc[df['reviewerID'] == x, 'asin']
#     list = ser.tolist()
#     transactions.append(list)

Eine For-Schleife ist sehr **leistungsaufwendig** und **ineffizient**, was für einen Datensatz von dieser Größe zu Berechnungen von mehreren Stunden führt.

### Gruppierung der Einträge mit GroupBy

In [11]:
#Input: DataFrame, Return: List
def grouped_to_transactions(df):
    # group = df.groupby('reviewerID')

    # transactions = []
    # for elem in group.groups.values():
    #     liste = []
    #     for ele in elem:
    #         liste.append(ele)
    #     transactions.append(liste)
    # print('Anzahl an Transaktionen: ', len(transactions))

    series = df.groupby('reviewerID')['asin'].apply(list)       #Elemente nach reviewerID gruppieren und in einer Liste speichern
    transactions = series.values.tolist()                       #Umwandeln der Serie in eine Liste

    print('Anzahl an Transaktionen: ', len(transactions))

    return transactions

transactions_1 = grouped_to_transactions(df_assoziation_red_1)

Anzahl an Transaktionen:  97550


In der Liste **'transactions_1'** ist jedes Element eine Liste, welche **alle Einkäufe eines Kunden innerhalb der Zeitscheibe** enthält. 

### Bereinigung der Transaktionen

In [12]:
#Duplikate innerhalb einer Transaktion entfernen

def remove_duplicates(transactions):
    no_dups = []
    counter_no = 0
    counter_yes = 0

    for elem in transactions:
        hold = list(set(elem))
        if hold == elem:
            # print('no Dups')
            counter_no = counter_no + 1
            no_dups.append(elem)
        else:
            # print('Dups')
            counter_yes = counter_yes + 1
            no_dups.append(hold)

    print('Transaktionen ohne Duplikate: ', counter_no)
    print('Transaktionen mit Duplikaten: ', counter_yes)    

    return no_dups

transactions_1 = remove_duplicates(transactions_1)

Transaktionen ohne Duplikate:  49760
Transaktionen mit Duplikaten:  47790


Innerhalb einer Transaktion können Elemente **mehrmals auftauchen**, sprich die Person hat ein Artikel mehrmals gekauft.\
Duplikate sind verzichtbar für das One-Hot-Encoding später und nehmen Speicherplatz weg.\
Deswegen werden die doppelten Elemente entfernt in diesem Schritt.


In [13]:
# Transaktionen aus 1 Element entfernen

def remove_single_items(transactions):
    print('Alle Transaktionen: ', len(transactions))
    counter = 0
    for elem in transactions:
        if len(elem) == 1:
            transactions.remove(elem)
            counter = counter + 1
    print('entfernte Transaktionen: ', counter)
    print('=> Nur Transaktionen mit mehr als 1 Artikel: ', len(transactions))    
    return transactions

transactions_1 = remove_single_items(transactions_1)

Alle Transaktionen:  97550
entfernte Transaktionen:  28869
=> Nur Transaktionen mit mehr als 1 Artikel:  68681


**13% der Transaktionen wurden entfernt**, weil sie nur aus 1 Artikel bestanden. Diese bringen nichts für die Assoziations Analyse.

In [14]:
print('Beispieltransaktion: ', transactions_1[0])

Beispieltransaktion:  ['B0050Z5A6E', 'B000E59A8E', 'B004DPNUAK', 'B002BDOHNA', 'B00YFU1QMW']


### TransactionEncoder

In [15]:
te_1 = TransactionEncoder()
te_ary_1 = te_1.fit(transactions_1).transform(transactions_1)
# te_ary.astype('int')

df_ohe_1 = pd.DataFrame(te_ary_1, columns=te_1.columns_)
# print(df_ohe.values.sum())

df_ohe_1

,0000032034,0899332757,0899333257,0971100764,1926644425,3843518912,7245456275,7245456313,7245456321,7500670133,...,B01GUBKFSO,B01GUKDFLY,B01GW32UWO,B01GWEKSGI,B01H3ZQV48,B01H4UICZ8,B01H7W8TI8,B01HGWE1KE,B01HHB2HK0,B01HHGOTAG
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68676,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
68677,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
68678,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
68679,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Prüfen ob das One-Hot-Encoding richtig abgelaufen ist.

In [16]:
df_ohe_1.iloc[0].value_counts()

False    52792
True         5
Name: 0, dtype: int64

Die 5 True Werte stimmen mit der obigen Beispieltransaktion überein.

### Apriori

In [17]:
apriori(df_ohe_1, min_support=0.001, use_colnames=True)

,support,itemsets
0,0.001805,(7245456313)
1,0.001194,(B00008BFYG)
2,0.002053,(B0000WR6W8)
3,0.001179,(B00012M5MS)
4,0.001150,(B00029PJZ0)
...,...,...
220,0.003291,"(B000VAPCU2, B00RM8I0LE, B00SK74C5Y, B00TZ900I4)"
221,0.003291,"(B000VAPCU2, B00SK74C5Y, B00SI9UB7M, B00TZ900I4)"
222,0.001849,"(B00CLVQNT8, B00TQ4QKIM, B001GFFRP6, B0049UXD7O)"
223,0.003334,"(B00RM8I0LE, B00SK74C5Y, B00SI9UB7M, B00TZ900I4)"


### FP-Growth

In [18]:
fpgrowth(df_ohe_1, min_support=0.001, use_colnames=True) 

,support,itemsets
0,0.001776,(B002BDOHNA)
1,0.001121,(B00KFTCCRA)
2,0.002271,(B006IL02OY)
3,0.002315,(B001WJ67S2)
4,0.001776,(B00C88T12K)
...,...,...
220,0.002068,"(B00QR2HBRA, B00125XCNY)"
221,0.002068,"(B00125XCNY, B00KXCFRSK)"
222,0.002068,"(B00QR2HBRA, B00125XCNY, B00KXCFRSK)"
223,0.001587,"(B0146A4SWA, B0013092CS)"


FP-Growth ist geeigneter für große Datensätze und kann diese effizienter durchgehen. Deswegen wird im weiteren Verlauf nur mit FP-Growth gearbeitet, damit möglichst viele Daten genutzt werden können. 

### Assoziations Regeln

In [19]:
frequent_itemsets_1 = fpgrowth(df_ohe_1, min_support=0.001, use_colnames=True)
frequent_itemsets_1['length'] = frequent_itemsets_1['itemsets'].apply(lambda x: len(x))
frequent_itemsets_1[(frequent_itemsets_1['length'] >= 2) & (frequent_itemsets_1['length'] >= 0.2)]

,support,itemsets,length
162,0.001689,"(B0015LT03G, B00136X6VU)",2
163,0.001674,"(B0015LY0DG, B0015LT03G)",2
164,0.001674,"(B0015LY0DG, B00136X6VU)",2
165,0.001674,"(B0015LY0DG, B0015LT03G, B00136X6VU)",3
166,0.001427,"(B00079ULA8, B0015LT03G)",2
...,...,...,...
220,0.002068,"(B00QR2HBRA, B00125XCNY)",2
221,0.002068,"(B00125XCNY, B00KXCFRSK)",2
222,0.002068,"(B00QR2HBRA, B00125XCNY, B00KXCFRSK)",3
223,0.001587,"(B0146A4SWA, B0013092CS)",2


In [20]:
rules_1 = association_rules(frequent_itemsets_1, metric = 'confidence', min_threshold=0.6)
rules_1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(B0015LT03G),(B00136X6VU),0.001689,0.001733,0.001689,1.000000,577.151261,0.001686,inf
1,(B00136X6VU),(B0015LT03G),0.001733,0.001689,0.001689,0.974790,577.151261,0.001686,39.599671
2,(B0015LY0DG),(B0015LT03G),0.001674,0.001689,0.001674,1.000000,592.077586,0.001672,inf
3,(B0015LT03G),(B0015LY0DG),0.001689,0.001674,0.001674,0.991379,592.077586,0.001672,115.805769
4,(B0015LY0DG),(B00136X6VU),0.001674,0.001733,0.001674,1.000000,577.151261,0.001672,inf
...,...,...,...,...,...,...,...,...,...
313,(B00KXCFRSK),"(B00QR2HBRA, B00125XCNY)",0.002781,0.002068,0.002068,0.743455,359.586387,0.002062,3.889900
314,(B0146A4SWA),(B0013092CS),0.001587,0.001602,0.001587,1.000000,624.372727,0.001585,inf
315,(B0013092CS),(B0146A4SWA),0.001602,0.001587,0.001587,0.990909,624.372727,0.001585,109.825425
316,(B0093IQT5A),(B00X6ZKQA0),0.001136,0.001107,0.001107,0.974359,880.525641,0.001105,38.956844


Eine genauere Analyse der Regeln findet am Ende des Notebooks statt.

## 2. Ansatz: alle Einkäufe von einer Auswahl von Kunden

In [21]:
print('Anzahl an Kunden: ', len(df['reviewerID'].unique()))

Anzahl an Kunden:  332447


### Kunden auswählen

In [22]:
customers = df_assoziation.groupby(['reviewerID'])['reviewerID'].count().sort_values(ascending=False)
customers = customers[:20000]

customer_list = customers.index.tolist()

print('Anzahl gekaufte Items: ', customers.values.sum())
print('Anzahl an Kunden: ', len(customers.index))


Anzahl gekaufte Items:  499885
Anzahl an Kunden:  20000


Von den **332447 Kunden** werden nur die **ersten 20000 Personen** betrachtet, welche am meisten gekauft haben, etwa **6% aller Kunden**. Diese haben ungefähr **17% aller Transaktionen** getätigt. 

### Reduzierung des Dataframes auf die Kunden

In [23]:
print('Vor der Reduzierung: ', df.shape)
df_assoziation_2 = df_assoziation[df_assoziation['reviewerID'].isin(customer_list)]
df_assoziation_2 = df_assoziation_2[['reviewerID', 'asin']]
print('Nach der Reduzierung: ', df_assoziation_2.shape)

Vor der Reduzierung:  (2839940, 12)
Nach der Reduzierung:  (499885, 2)


Es wurden etwa **2,5 Millionen Einträge verworfen**.

### Gruppierung der Einträge mit GroupBy

In [24]:
transactions_2 = grouped_to_transactions(df_assoziation_2)

Anzahl an Transaktionen:  20000


### Bereinigung der Transaktionen

In [25]:
#Duplikate innerhalb einer Transaktion entfernen
transactions_2 = remove_duplicates(transactions_2)

Transaktionen ohne Duplikate:  0
Transaktionen mit Duplikaten:  20000


In [26]:
# Transaktionen aus 1 Element entfernen
transactions_2 = remove_single_items(transactions_2)

Alle Transaktionen:  20000
entfernte Transaktionen:  3
=> Nur Transaktionen mit mehr als 1 Artikel:  19997


### TransactionEncoder

In [27]:
te_2 = TransactionEncoder()
te_ary_2 = te_2.fit(transactions_2).transform(transactions_2)
# te_ary.astype('int')

df_ohe_2 = pd.DataFrame(te_ary_2, columns=te_2.columns_)
# df_ohe.values.sum()

df_ohe_2

,0899332757,0899333257,0971100764,1620878747,7245456275,7245456313,7500670133,9641766805,9877000490,B000002NUS,...,B01HJ2HYIW,B01HJ3MMSI,B01HJ4DN08,B01HJ4EYS8,B01HJ8YC0Y,B01HJA241G,B01HJDHNX2,B01HJDZ34I,B01HJGAJ9O,B01HJHHBHG
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19992,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19993,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19994,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Assoziations Regeln

In [28]:
frequent_itemsets_2 = fpgrowth(df_ohe_1, min_support=0.001, use_colnames=True)
frequent_itemsets_2['length'] = frequent_itemsets_2['itemsets'].apply(lambda x: len(x))
frequent_itemsets_2[(frequent_itemsets_2['length'] >= 2) & (frequent_itemsets_1['support'] > 0.003)]

,support,itemsets,length
177,0.003334,"(B00SK74C5Y, B00TZ900I4)",2
178,0.003334,"(B00SK74C5Y, B00SI9UB7M)",2
179,0.003334,"(B00TZ900I4, B00SI9UB7M)",2
180,0.003334,"(B00SK74C5Y, B00SI9UB7M, B00TZ900I4)",3
181,0.003334,"(B00RM8I0LE, B00SI9UB7M)",2
182,0.003334,"(B00RM8I0LE, B00SK74C5Y)",2
183,0.003334,"(B00RM8I0LE, B00TZ900I4)",2
184,0.003334,"(B00RM8I0LE, B00SK74C5Y, B00SI9UB7M)",3
185,0.003334,"(B00RM8I0LE, B00SI9UB7M, B00TZ900I4)",3
186,0.003334,"(B00RM8I0LE, B00SK74C5Y, B00TZ900I4)",3


In [29]:
rules_2 = association_rules(frequent_itemsets_2, metric = 'confidence', min_threshold=0.6)
rules_2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(B0015LT03G),(B00136X6VU),0.001689,0.001733,0.001689,1.000000,577.151261,0.001686,inf
1,(B00136X6VU),(B0015LT03G),0.001733,0.001689,0.001689,0.974790,577.151261,0.001686,39.599671
2,(B0015LY0DG),(B0015LT03G),0.001674,0.001689,0.001674,1.000000,592.077586,0.001672,inf
3,(B0015LT03G),(B0015LY0DG),0.001689,0.001674,0.001674,0.991379,592.077586,0.001672,115.805769
4,(B0015LY0DG),(B00136X6VU),0.001674,0.001733,0.001674,1.000000,577.151261,0.001672,inf
...,...,...,...,...,...,...,...,...,...
313,(B00KXCFRSK),"(B00QR2HBRA, B00125XCNY)",0.002781,0.002068,0.002068,0.743455,359.586387,0.002062,3.889900
314,(B0146A4SWA),(B0013092CS),0.001587,0.001602,0.001587,1.000000,624.372727,0.001585,inf
315,(B0013092CS),(B0146A4SWA),0.001602,0.001587,0.001587,0.990909,624.372727,0.001585,109.825425
316,(B0093IQT5A),(B00X6ZKQA0),0.001136,0.001107,0.001107,0.974359,880.525641,0.001105,38.956844


## 3. Ansatz: am häufigsten gekauftes Item und zugehörige Transaktionen

### meistgekaufteste Item bestimmen

In [30]:
items = []
for index, row in df_assoziation.iterrows():
    items.append(row['asin'])
counter = Counter(items)
most_common = counter.most_common(5)
print(most_common)

[('B0010O748Q', 3845), ('B0012Q2S4W', 3205), ('B0015LY0DG', 3072), ('B0015LT03G', 3072), ('B0014VX2M2', 3046)]


In [31]:
a = df_meta.loc[df_meta['asin'] == 'B0010O748Q']
list(a['title'])

['SE FS374 All-Weather Emergency 2-IN-1 Fire Starter and Magnesium Fuel Bar, Flint and Steel Kit']

Ein **Feuerzeug-Set** wurde mit **3845** Einkäufen am häufigsten eingekauft.

### Dataframe auf dieses Item und weitere Einkäufe dieser Kunden reduzieren

Zuerst wird ein DataFrame erstellt, welches alle Transaktionen von diesem Feuerzeug enthält.

In [32]:
df_most_item = df_assoziation.loc[df_assoziation['asin'] == 'B0010O748Q']
df_most_item.shape


(3845, 12)

Danach werden die Kunden-Ids abgespeichert, welche das Feuerzeug gekauft haben.

In [33]:
customer_list = df_most_item['reviewerID'].unique()
len(customer_list)

1924

Das Item wurde zwar **3845-mal gekauft**, aber nur von **1924 verschiedenen Benutzern**. Einige haben es also mehrfach gekauft, was bei einem Feuerzeug auch Sinn macht.\
Von den Kunden aus 'customer_list' werden nun alle Transaktionen geholt. Damit wird später **in jedem Warenkorb mindestens 1 gleiches Item** existieren.

In [34]:
df_assoziation_3 = df_assoziation[df_assoziation['reviewerID'].isin(customer_list)]

### Gruppierung der Einträge mit GroupBy

In [35]:
transactions_3 = grouped_to_transactions(df_assoziation_3)

Anzahl an Transaktionen:  1924


### Bereinigung der Transaktionen

In [36]:
#Duplikate innerhalb einer Transaktion entfernen
transactions_3 = remove_duplicates(transactions_3)

Transaktionen ohne Duplikate:  0
Transaktionen mit Duplikaten:  1924


In [37]:
transactions_3 = remove_single_items(transactions_3)

Alle Transaktionen:  1924
entfernte Transaktionen:  2
=> Nur Transaktionen mit mehr als 1 Artikel:  1922


### TransactionEncoder

In [38]:
te_3 = TransactionEncoder()
te_ary_3 = te_3.fit(transactions_3).transform(transactions_3)
# te_ary.astype('int')

df_ohe_3 = pd.DataFrame(te_ary_3, columns=te_3.columns_)
# df_ohe.values.sum()

df_ohe_3

,0971100764,1620878747,7245456275,7245456313,7245456453,B00003G4JR,B00004TBLW,B00004U9IY,B000051ZHS,B0000568SY,...,B01HGUKZQU,B01HGV8R50,B01HHB2HK0,B01HHLDYJI,B01HI69CB6,B01HIGS3O8,B01HIIQ9PQ,B01HIN1KUU,B01HJ234E0,B01HJ4EYS8
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1918,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1919,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1920,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Assoziations Regeln

In [39]:
frequent_itemsets_3 = fpgrowth(df_ohe_3, min_support=0.001, use_colnames=True)
frequent_itemsets_3['length'] = frequent_itemsets_3['itemsets'].apply(lambda x: len(x))
frequent_itemsets_3[(frequent_itemsets_3['length'] >= 2) & (frequent_itemsets_3['support'] > 0.005)]

,support,itemsets,length
2973,0.042144,"(B0026OOS60, B0010O748Q)",2
2974,0.008845,"(B0026OOS60, B001949TKS)",2
2975,0.006764,"(B0026OOS60, B001ID4ZY0)",2
2976,0.009886,"(B0026OOS60, B000GCRWCG)",2
2977,0.008845,"(B0026OOS60, B0010O748Q, B001949TKS)",3
...,...,...,...
8920,0.006243,"(B0010O748Q, B000XH5H6W)",2
9095,0.005203,"(B0010O748Q, B0027J4YMM)",2
9226,0.005203,"(B000UPOG28, B0010O748Q)",2
9547,0.005203,"(B0010O748Q, B001EI7578)",2


In [40]:
rules_3 = association_rules(frequent_itemsets_3, metric = 'lift', min_threshold=2)
rules_3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(B002BDOHNA),(B001MYEU0E),0.003122,0.003122,0.001041,0.333333,106.777778,0.001031,1.495317
1,(B001MYEU0E),(B002BDOHNA),0.003122,0.003122,0.001041,0.333333,106.777778,0.001031,1.495317
2,(B0013RD6OQ),(B002BDOHNA),0.004683,0.003122,0.001041,0.222222,71.185185,0.001026,1.281701
3,(B002BDOHNA),(B0013RD6OQ),0.003122,0.004683,0.001041,0.333333,71.185185,0.001026,1.492976
4,"(B0010O748Q, B002BDOHNA)",(B001MYEU0E),0.003122,0.003122,0.001041,0.333333,106.777778,0.001031,1.495317
...,...,...,...,...,...,...,...,...,...
56983,(B001ID4ZY0),(B00C2924LU),0.071280,0.001041,0.001041,0.014599,14.029197,0.000966,1.013759
56984,"(B00C2924LU, B0010O748Q)",(B001ID4ZY0),0.001041,0.071280,0.001041,1.000000,14.029197,0.000966,inf
56985,"(B0010O748Q, B001ID4ZY0)",(B00C2924LU),0.071280,0.001041,0.001041,0.014599,14.029197,0.000966,1.013759
56986,(B00C2924LU),"(B0010O748Q, B001ID4ZY0)",0.001041,0.071280,0.001041,1.000000,14.029197,0.000966,inf


Hier ist auffällig, dass auf den ersten Blick 2 Regeln eine **Confidence = 1** haben. Diese werden später genauer angeschaut.

## 4. Ansatz: Metadaten - häufigste Kategorie

Auf den ersten Blick haben viele Produkte keine Einträge in der Spalte 'also_buy'. Diese können also verworfen werden. 

In [41]:
df_meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Adult Tutu Assorted Colors (Turquoise),[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (","[B071LQWQBQ, B00M14DG0O, B076GTTY9W, B07BSZ4GY...","<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.80,0000032042,[],[],NaN
1,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Bububibi Adult Ballet Tutu Cheetah Pink,[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (",[],"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.97,0000032069,[],[],NaN
2,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Girls Ballet Tutu Neon Orange,[B01MYHD3LV],,Unknown,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#924,198 in Toys & Games (See Top 100 in Toy...","[B0152HBC52, B07C29WWV1, B01M0BIP9H, B00VXJHFR...",Toys & Games,,,$5.70,0000031860,[],[],NaN
3,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,0000031852,[],[],NaN
4,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[Dance tutu for girls ages 2-8 years. Perfect ...,,Girls Ballet Tutu Neon Blue,"[B009THXHPA, B00BEU1ZBI, B079PYLZTM, B00BEU1ZR...",,Bububibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#393,597 in Toys & Games (See Top 100 in Toy...","[B009THXHPA, B079PYLZTM, B079PQPLT3, B00UP6A3R...",Toys & Games,,,$6.54,0000031895,[],[],NaN


In [42]:
df_meta_red = df_meta[['asin', 'also_buy', 'category']]

transactions_4 = []
category = []

counter = 0

for index, row in df_meta_red.iterrows():

    liste = []
    if row['also_buy'] != []:               # wenn es Einträge in 'also_buy' gibt           
        liste.append(row['asin'])
        for y in row['also_buy']:
            liste.append(y)
        transactions_4.append(liste)

        category.extend(row['category'])

print('Anzahl an Artikeln', len(df_meta_red.index))
print('Entfernte Artikel:', (len(df_meta_red.index) - len(transactions_4)))
print('=> Artikel mit weiteren Einkäufen' , len(transactions_4))

Anzahl an Artikeln 962300
Entfernte Artikel: 696170
=> Artikel mit weiteren Einkäufen 266130


Diese Anzahl an Transaktionen ist wieder zu viel für die weitere Berechnung. Ein Weg um die Anzahl an Artikel zu reduzieren ist es nur Artikel von einer Kategorie zu betrachten und die zugehörogen 'also_buy' Artikel.

### Häufigste Kategorien

In [43]:
counter = Counter(category)
most_common = counter.most_common(5)
print(most_common)

[('Sports & Outdoors', 251946), ('Sports & Fitness', 156977), ('Outdoor Recreation', 62267), ('Hunting & Fishing', 59290), ('Clothing', 39855)]


### Alle 'Sports & Outdoors' Artikel wählen

In [44]:
def transactions_by_category(df, category):
    transactions = []

    for index, row in df.iterrows():
        if category in row['category']:
            liste = []
            if row['also_buy'] != []:               # wenn es Einträge in 'also_buy' gibt           
                liste.append(row['asin'])
                for y in row['also_buy']:
                    liste.append(y)
                transactions.append(liste)

    print('=> valide Artikel aus Sports & Outdoors' , len(transactions))
    return transactions

transactions_5 = transactions_by_category(df_meta_red, 'Sports & Outdoors')

=> valide Artikel aus Sports & Outdoors 251946


In [ ]:
# te_5 = TransactionEncoder()
# te_ary_5 = te_5.fit(transactions_5).transform(transactions_5)
# # te_ary.astype('int')

# df_ohe_5 = pd.DataFrame(te_ary_5, columns=te_5.columns_)
# # df_ohe.values.sum()

# apriori(df_ohe_5, min_support=0.005, use_colnames=True)

# ==> 222 GB benötigt für 251946 Transaktionen

Aufgrund der großen Menge an Daten wird zu viel Speicherplatz benötigt. Für den weiteren Verlauf wird eine zufällige Stichprobe der Daten ausgewählt.

### Sample aus dem Datensatz wählen

In [45]:
df_meta_sample = df_meta.sample(frac=0.20, replace=False, random_state=1)
df_meta_sample.shape

(192460, 19)

Wir nehmen nur **20%** des Datensatzes

### Mögliche Kategorien identifizieren

In [46]:
category = []
for index, row in df_meta_sample.iterrows():
    if row['also_buy'] != []:
        category.extend(row['category'])

counter = Counter(category)
most_common = counter.most_common(5)
print(most_common)

[('Sports & Outdoors', 50627), ('Sports & Fitness', 31608), ('Outdoor Recreation', 12488), ('Hunting & Fishing', 11961), ('Clothing', 7885)]


Da bereits im gesamten Datensatz 'Sports & Outdoors' am häufigsten vertreten war, ist es auch mit einer hohen Wahrscheinlichkeit im Sample stark vertreten.

### Nur Items aus der Kategorie 'Sports & Outdoors'

In [47]:
transactions_6 = transactions_by_category(df_meta_sample, 'Sports & Outdoors')

=> valide Artikel aus Sports & Outdoors 50627


### TransactionEncoder

In [48]:
te_4 = TransactionEncoder()
te_ary_4 = te_4.fit(transactions_6).transform(transactions_6)
# te_ary.astype('int')

df_ohe_4 = pd.DataFrame(te_ary_4, columns=te_4.columns_)
# df_ohe.values.sum()

df_ohe_4

,0000031887,0007145160,0007245173,0007333447,0007424604,0007494823,0007577311,0007921268,0008133786,0008201080,...,B07MSQ3QJC,B07MSQSJ9L,B07MSR8QWN,B07MSWL7ZX,B07MSWTK9P,B07MSZRG8X,B07MT3J5PH,B07MT4411X,B07MT62D8N,BT00DC6QU4
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50622,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
50623,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
50624,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
50625,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### FP-Growth

In [49]:
frequent_itemsets_4 = fpgrowth(df_ohe_4, min_support=0.004, use_colnames=True)
frequent_itemsets_4['length'] = frequent_itemsets_4['itemsets'].apply(lambda x: len(x))
frequent_itemsets_4[(frequent_itemsets_4['length'] >= 2)]

,support,itemsets,length
99,0.004800,"(B000BYAKDM, B0012AXR4S)",2
100,0.004168,"(B0012AXR4S, B004TAB7ZO)",2
101,0.005491,"(B00C6OUDX2, B005ADORGK)",2
102,0.005392,"(B0094FP958, B005ADORGK)",2
103,0.004820,"(B00C6OUDX2, B0094FP958)",2
104,0.004069,"(B00C6OUDX2, B0094FP958, B005ADORGK)",3
105,0.004266,"(B000NORVHI, B002IDZXRM)",2
106,0.004997,"(B017KU1DIK, B00H9IFSH6)",2
107,0.004266,"(B000HZGTUS, B00H9IFSH6)",2
108,0.004859,"(B002IDZXRM, B002VYB4QC)",2


In [50]:
rules_4 = association_rules(frequent_itemsets_4, metric = 'confidence', min_threshold=0.6)
rules_4

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(B000BYAKDM),(B0012AXR4S),0.006222,0.011535,0.004800,0.771429,66.875196,0.004728,4.324533
1,(B004TAB7ZO),(B0012AXR4S),0.004938,0.011535,0.004168,0.844000,73.166418,0.004111,6.336312
2,(B0094FP958),(B005ADORGK),0.007921,0.009777,0.005392,0.680798,69.629819,0.005315,3.102182
3,(B0094FP958),(B00C6OUDX2),0.007921,0.009521,0.004820,0.608479,63.911735,0.004744,2.529823
4,"(B00C6OUDX2, B0094FP958)",(B005ADORGK),0.004820,0.009777,0.004069,0.844262,86.348419,0.004022,6.358271
5,"(B00C6OUDX2, B005ADORGK)",(B0094FP958),0.005491,0.007921,0.004069,0.741007,93.553544,0.004025,3.830529
6,"(B0094FP958, B005ADORGK)",(B00C6OUDX2),0.005392,0.009521,0.004069,0.754579,79.257383,0.004018,4.035834
7,(B017KU1DIK),(B00H9IFSH6),0.008039,0.007861,0.004997,0.621622,79.072457,0.004934,2.622081
8,(B00H9IFSH6),(B017KU1DIK),0.007861,0.008039,0.004997,0.635678,79.072457,0.004934,2.722761
9,(B002VYB4QC),(B002IDZXRM),0.007842,0.008612,0.004859,0.619647,71.951575,0.004792,2.606497


## Untersuchung der erstellten Assozations Regeln

Im genaueren werden die erstellten Regeln aus **3. Ansatz: am häufigsten gekauftes Item und zugehörige Transaktionen** und **4. Ansatz: Metadaten betrachtet**.\
Zuletzt werden alle Regeln gemeinsam betrachtet.

### 3. Ansatz: am häufigsten gekauftes Item und zugehörige Transaktionen

Hier waren Regeln mit einer Confidence von 1. Diese sollen nun genauer betrachtet werden.

In [51]:
rules_3[rules_3['confidence'] == 1]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,"(B0013RD6OQ, B002BDOHNA)",(B001MYEU0E),0.001041,0.003122,0.001041,1.0,320.333333,0.001037,inf
14,"(B002BDOHNA, B001MYEU0E)",(B0013RD6OQ),0.001041,0.004683,0.001041,1.0,213.555556,0.001036,inf
18,"(B0010O748Q, B0013RD6OQ, B002BDOHNA)",(B001MYEU0E),0.001041,0.003122,0.001041,1.0,320.333333,0.001037,inf
20,"(B0010O748Q, B002BDOHNA, B001MYEU0E)",(B0013RD6OQ),0.001041,0.004683,0.001041,1.0,213.555556,0.001036,inf
24,"(B0013RD6OQ, B002BDOHNA)","(B0010O748Q, B001MYEU0E)",0.001041,0.003122,0.001041,1.0,320.333333,0.001037,inf
...,...,...,...,...,...,...,...,...,...
56980,(B01EIQH9JG),"(B0010O748Q, B0046IEC2E)",0.001041,0.001041,0.001041,1.0,961.000000,0.001039,inf
56981,(B0046IEC2E),"(B0010O748Q, B01EIQH9JG)",0.001041,0.001041,0.001041,1.0,961.000000,0.001039,inf
56982,(B00C2924LU),(B001ID4ZY0),0.001041,0.071280,0.001041,1.0,14.029197,0.000966,inf
56984,"(B00C2924LU, B0010O748Q)",(B001ID4ZY0),0.001041,0.071280,0.001041,1.0,14.029197,0.000966,inf


Mit 26937 Regeln mit einer Confidence = 1 lässt sich vermuten, dass der ausgewählte Datensatz hier zu Problemen geführt hat. Dadurch, dass in jedem Warenkorb das Feuerzeug ist, werden Beziehungen damit immer eine Condifence von 1 haben. An dem geringen Support ist zwar erkennbar, dass nur wenige Warenkörbe diese Kombination haben, aber trotzdem eine perfekte Confidence. Wenn die Kombination von Artikel auftritt, dann taucht auch immer das Feuerzeug auf.

Es gibt aber noch weitere Regeln mit Confidence = 1, welche nicht das Feuerzeug beinhalten. Diese wollen wir nun anschauen.

#### Umwandlung der 'asin' in 'title'

In [56]:
def translate_asin(df):
    for index, row in df.iterrows():
        frozen = list(row['antecedents'])                              # Frozenset in Liste umwandeln 
        if len(frozen) == 1:                                            # wenn nur 1 Eintrag in der Zelle ist
            x = df_meta.loc[df_meta['asin'] == frozen[0]]['title']
            if x.values.size > 0:                                       # Prüfen ob der Asin in df_meta drin ist
                df.at[index, 'antecedents'] = x

        elif len(frozen) > 1:                                           # wenn mehr als 1 Eintrag in der Zelle sind
            names = []
            for elem in frozen:
                x = df_meta.loc[df_meta['asin'] == elem]['title']
                if x.values.size > 0:
                    names.append(x)
                else:
                    names.append(elem)
                df.at[index, 'antecedents'] = names

        frozen = list(row['consequents'])                              # Frozenset in Liste umwandeln 
        if len(frozen) == 1:                                            # wenn nur 1 Eintrag in der Zelle ist
            x = df_meta.loc[df_meta['asin'] == frozen[0]]['title']
            if x.values.size > 0:                                       # Prüfen ob der Asin in df_meta drin ist
                df.at[index, 'consequents'] = x

        elif len(frozen) > 1:                                           # wenn mehr als 1 Eintrag in der Zelle sind
            names = []
            for elem in frozen:
                x = df_meta.loc[df_meta['asin'] == elem]['title']
                if x.values.size > 0:
                    names.append(x)
                else:
                    names.append(elem)
                df.at[index, 'consequents'] = names
    return df

    # Auf alle 58000 Zeilen braucht der Code sehr lange, weil er ineffizient ist

rules_3 = translate_asin(rules_3)
        

KeyboardInterrupt: 

In [57]:
rules_3_1 = rules_3[rules_3['confidence'] == 1]
rules_3_1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,[[MTM Universal Ammo Loading Tray Red (include...,105727 Lyman 49th Edition Reloading Handboo...,0.001041,0.003122,0.001041,1.0,320.333333,0.001037,inf
14,[[Frankford Arsenal DS-750 Digital Reloading S...,66786 MTM Universal Ammo Loading Tray Red (...,0.001041,0.004683,0.001041,1.0,213.555556,0.001036,inf
18,[[SE FS374 All-Weather Emergency 2-IN-1 Fire S...,105727 Lyman 49th Edition Reloading Handboo...,0.001041,0.003122,0.001041,1.0,320.333333,0.001037,inf
20,[[SE FS374 All-Weather Emergency 2-IN-1 Fire S...,66786 MTM Universal Ammo Loading Tray Red (...,0.001041,0.004683,0.001041,1.0,213.555556,0.001036,inf
24,[[MTM Universal Ammo Loading Tray Red (include...,[[SE FS374 All-Weather Emergency 2-IN-1 Fire S...,0.001041,0.003122,0.001041,1.0,320.333333,0.001037,inf
...,...,...,...,...,...,...,...,...,...
56980,(B01EIQH9JG),"(B0010O748Q, B0046IEC2E)",0.001041,0.001041,0.001041,1.0,961.000000,0.001039,inf
56981,(B0046IEC2E),"(B0010O748Q, B01EIQH9JG)",0.001041,0.001041,0.001041,1.0,961.000000,0.001039,inf
56982,(B00C2924LU),(B001ID4ZY0),0.001041,0.071280,0.001041,1.0,14.029197,0.000966,inf
56984,"(B00C2924LU, B0010O748Q)",(B001ID4ZY0),0.001041,0.071280,0.001041,1.0,14.029197,0.000966,inf


| Antecedents                                                                                                                                                                                                                                                                                                                                    | Consequents                                                                                                                                                                                                                                              | Support | Confidence | Lift   |
|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------|------------|--------|
| TOUGH-GRID 750lb Paracord/Parachute Cord - Genuine Mil Spec Type IV 750lb Paracord Used by the US Military (MIl-C-5040-H) - 100% Nylon - Made In The USA.                                                                                                                                                                                      | SE FS374 All-Weather Emergency 2-IN-1 Fire Starter and Magnesium Fuel Bar, Flint and Steel Kit,TOUGH-GRID 750lb Paracord/Parachute Cord - Genuine Mil Spec Type IV 750lb Paracord Used by the US Military (MIl-C-5040-H) - 100% Nylon - Made In The USA. | 0.007   | 1          | 58.242 |
| Esbit 1300-Degree Smokeless Solid Fuel Tablets for Hobby, Outdoor, and Emergency Use, 20 Pieces Each 4g,Esbit 1300-Degree Smokeless Solid Fuel Tablets for Hobby, Outdoor, and Emergency Use, 20 Pieces Each 4g,The Friendly Swede Magnesium Flint Fire Starter - 3 Pack - Survival Firesteel Blocks with Striker, Camping Emergency Equipment | Chainmate CM-24SSP 24-Inch Survival Pocket Chain Saw With Pouch                                                                                                                                                                                          | 0.001   | 1          | 23.728 |
| The Friendly Swede Magnesium Flint Fire Starter - 3 Pack - Survival Firesteel Blocks with Striker, Camping Emergency Equipment,Chainmate CM-24SSP 24-Inch Survival Pocket Chain Saw With Pouch                                                                                                                                                 | Esbit 1300-Degree Smokeless Solid Fuel Tablets for Hobby, Outdoor, and Emergency Use, 20 Pieces Each 4g,Esbit 1300-Degree Smokeless Solid Fuel Tablets for Hobby, Outdoor, and Emergency Use, 20 Pieces Each 4g                                          | 0.001   | 1          | 39.224 |

In [58]:
rules_3_smaller1 = rules_3[(rules_3['confidence'] < 1) & (rules_3['support'] > 0.002)]
rules_3_smaller1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
96,126324 Chainmate CM-24SSP 24-Inch Survival ...,78099 Potable Aqua Water Purification Treat...,0.042144,0.083767,0.008845,0.209877,2.505483,0.005315,1.159608
97,78099 Potable Aqua Water Purification Treat...,126324 Chainmate CM-24SSP 24-Inch Survival ...,0.083767,0.042144,0.008845,0.105590,2.505483,0.005315,1.070937
98,126324 Chainmate CM-24SSP 24-Inch Survival ...,100312 SE CC4580 Military Lensatic and Pri...,0.042144,0.071280,0.006764,0.160494,2.251600,0.003760,1.106270
99,100312 SE CC4580 Military Lensatic and Pri...,126324 Chainmate CM-24SSP 24-Inch Survival ...,0.071280,0.042144,0.006764,0.094891,2.251600,0.003760,1.058277
100,126324 Chainmate CM-24SSP 24-Inch Survival ...,26775 Mylar Men's Emergency Thermal Blanket...,0.042144,0.078564,0.009886,0.234568,2.985692,0.006575,1.203812
...,...,...,...,...,...,...,...,...,...
46853,"(B0010O748Q, B0012Q2S4W)",(B00BMSGU9Y),0.004162,0.003122,0.003122,0.750000,240.250000,0.003109,3.987513
46855,(B0012Q2S4W),"(B0010O748Q, B00BMSGU9Y)",0.004162,0.003122,0.003122,0.750000,240.250000,0.003109,3.987513
51805,(B000BS05Y2),(B00B1G7CXK),0.006243,0.002601,0.002601,0.416667,160.166667,0.002585,1.709826
51812,"(B0010O748Q, B000BS05Y2)",(B00B1G7CXK),0.006243,0.002601,0.002601,0.416667,160.166667,0.002585,1.709826


| Antecedents                                                                                                                                                                                                                                                               | Consequents                                                                                                                                     | Support | Confidence | Lift   |
|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|---------|------------|--------|
| Esbit 1300-Degree Smokeless Solid Fuel Tablets for Hobby, Outdoor, and Emergency Use, 20 Pieces Each 4g,  Mylar Men's Emergency Thermal Blankets (10 Pack)                                                                                                                | Potable Aqua Water Purification Treatment (50 Tablets) - Portable Drinking Water Treatment Ideal for Emergencies, Survival, Travel, and Camping | 0.003   | 0.857      | 10.232 |
| Potable Aqua Water Purification Treatment (50 Tablets) - Portable Drinking Water Treatment Ideal for Emergencies, Survival, Travel, and Camping Grizzly Gear Emergency Survival Mylar Thermal Reflective Cold Weather Shelter Tube Tent - XL Size Fits 2 Adults - 8' X 3' | Chainmate CM-24SSP 24-Inch Survival Pocket Chain Saw With Pouch                                                                                 | 0.002   | 0.8        | 18.983 |
| Rothco 3 Piece Canteen Kit with Cover &amp; Aluminum Cup SE CC4580 Military Lensatic and Prismatic Sighting Survival Emergency Compass with Pouch                                                                                                                         | Potable Aqua Water Purification Treatment (50 Tablets) - Portable Drinking Water Treatment Ideal for Emergencies, Survival, Travel, and Camping | 0.002   | 0.667      | 7.959  |

### Ansatz 4: Metadaten

In [59]:
rules_4

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(B000BYAKDM),(B0012AXR4S),0.006222,0.011535,0.004800,0.771429,66.875196,0.004728,4.324533
1,(B004TAB7ZO),(B0012AXR4S),0.004938,0.011535,0.004168,0.844000,73.166418,0.004111,6.336312
2,(B0094FP958),(B005ADORGK),0.007921,0.009777,0.005392,0.680798,69.629819,0.005315,3.102182
3,(B0094FP958),(B00C6OUDX2),0.007921,0.009521,0.004820,0.608479,63.911735,0.004744,2.529823
4,"(B00C6OUDX2, B0094FP958)",(B005ADORGK),0.004820,0.009777,0.004069,0.844262,86.348419,0.004022,6.358271
5,"(B00C6OUDX2, B005ADORGK)",(B0094FP958),0.005491,0.007921,0.004069,0.741007,93.553544,0.004025,3.830529
6,"(B0094FP958, B005ADORGK)",(B00C6OUDX2),0.005392,0.009521,0.004069,0.754579,79.257383,0.004018,4.035834
7,(B017KU1DIK),(B00H9IFSH6),0.008039,0.007861,0.004997,0.621622,79.072457,0.004934,2.622081
8,(B00H9IFSH6),(B017KU1DIK),0.007861,0.008039,0.004997,0.635678,79.072457,0.004934,2.722761
9,(B002VYB4QC),(B002IDZXRM),0.007842,0.008612,0.004859,0.619647,71.951575,0.004792,2.606497


#### Umwandlung der 'asin' in 'title'

In [60]:
# for index, row in rules_4.iterrows():
#     frozen = list(row['antecedents'])                              # Frozenset in Liste umwandeln 
#     if len(frozen) == 1:                                            # wenn nur 1 Eintrag in der Zelle ist
#         x = df_meta.loc[df_meta['asin'] == frozen[0]]['title']
#         if x.values.size > 0:                                       # Prüfen ob der Asin in df_meta drin ist
#             rules_4.at[index, 'antecedents'] = x

#     elif len(frozen) > 1:                                           # wenn mehr als 1 Eintrag in der Zelle sind
#         names = []
#         for elem in frozen:
#             x = df_meta.loc[df_meta['asin'] == elem]['title']
#             if x.values.size > 0:
#                 names.append(x)
#             else:
#                 names.append(elem)
#             rules_4.at[index, 'antecedents'] = names

#     frozen = list(row['consequents'])                              # Frozenset in Liste umwandeln 
#     if len(frozen) == 1:                                            # wenn nur 1 Eintrag in der Zelle ist
#         x = df_meta.loc[df_meta['asin'] == frozen[0]]['title']
#         if x.values.size > 0:                                       # Prüfen ob der Asin in df_meta drin ist
#             rules_4.at[index, 'consequents'] = x

#     elif len(frozen) > 1:                                           # wenn mehr als 1 Eintrag in der Zelle sind
#         names = []
#         for elem in frozen:
#             x = df_meta.loc[df_meta['asin'] == elem]['title']
#             if x.values.size > 0:
#                 names.append(x)
#             else:
#                 names.append(elem)
#             rules_4.at[index, 'consequents'] = names
    
rules_4 = translate_asin(rules_4)



In [64]:
rules_4.sort_values(by = 'confidence' , ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,"[B00C6OUDX2, B0094FP958]",(B005ADORGK),0.004820,0.009777,0.004069,0.844262,86.348419,0.004022,6.358271
1,(B004TAB7ZO),(B0012AXR4S),0.004938,0.011535,0.004168,0.844000,73.166418,0.004111,6.336312
17,64506 Finish Line Fiber Grip Carbon Fiber B...,455767 Park Tool MLP- 1.2 Chain Plier Name:...,0.005550,0.009145,0.004346,0.782918,85.608633,0.004295,4.564429
0,19295 Wheeler Engineering Level Level Level...,(B0012AXR4S),0.006222,0.011535,0.004800,0.771429,66.875196,0.004728,4.324533
15,(B07D7VTQ1R),(B00U8TFZEY),0.006202,0.006419,0.004741,0.764331,119.063988,0.004701,4.216004
6,"[B0094FP958, B005ADORGK]",(B00C6OUDX2),0.005392,0.009521,0.004069,0.754579,79.257383,0.004018,4.035834
5,"[B00C6OUDX2, B005ADORGK]",(B0094FP958),0.005491,0.007921,0.004069,0.741007,93.553544,0.004025,3.830529
16,(B00U8TFZEY),(B07D7VTQ1R),0.006419,0.006202,0.004741,0.738462,119.063988,0.004701,3.799815
14,(B00KQS2NGK),(B003DPVERM),0.007387,0.008671,0.005215,0.705882,81.404797,0.005151,3.370518
10,653675 100 Pcs Alloy Road Mountain Bicycle ...,234383 Shimano Universal Standard Brake Cab...,0.007170,0.012128,0.004938,0.688705,56.786775,0.004851,3.173430


| Antecedents                                                                    | Consequents                                                                                                                                                                          | Support | Confidence | Lift   |
|--------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------|------------|--------|
| Finish Line Fiber Grip Carbon Fiber Bicycle Assembly Gel                       | Park Tool MLP- 1.2 Chain Plier                                                                                                                                                       | 0.004   | 0.787      | 87.765 |
| 100 Pcs Alloy Road Mountain Bicycle Bike Brake Cable Cap End Tips Crimp Silver | Shimano Universal Standard Brake Cable Set, For MTB or Road Bikes                                                                                                                    | 0.004   | 0.689      | 89.315 |
| Topeak D2 SmartGauge                                                           | Park Tool MLP- 1.2 Chain Plier                                                                                                                                                       | 0.004   | 0.654      | 72.840 |
| 16g Threaded CO2 Cartridges 6-Pack Mole-Zap/Ant Zap Refills                    | Pro Bike Tool CO2 Inflator - Quick &amp; Easy - Presta and Schrader Valve Compatible - Bicycle Tire Pump for Road and Mountain Bikes - Insulated Sleeve - No CO2 Cartridges Included | 0.004   | 0.640      | 84.508 |

### Alle Regeln aus den verschiedenen Ansätzen

In [65]:
all_rules = pd.concat([rules_1, rules_2, rules_3, rules_4], axis = 0)

#### Umwandlung der 'asin' in 'title'

In [ ]:
# for index, row in all_rules.iterrows():
#     frozen = list(row['antecedents'])                              # Frozenset in Liste umwandeln 
#     if len(frozen) == 1:                                            # wenn nur 1 Eintrag in der Zelle ist
#         x = df_meta.loc[df_meta['asin'] == frozen[0]]['title']
#         if x.values.size > 0:                                       # Prüfen ob der Asin in df_meta drin ist
#             all_rules.at[index, 'antecedents'] = x

#     elif len(frozen) > 1:                                           # wenn mehr als 1 Eintrag in der Zelle sind
#         names = []
#         for elem in frozen:
#             x = df_meta.loc[df_meta['asin'] == elem]['title']
#             if x.values.size > 0:
#                 names.append(x)
#             else:
#                 names.append(elem)
#             all_rules.at[index, 'antecedents'] = names

#     frozen = list(row['consequents'])                              # Frozenset in Liste umwandeln 
#     if len(frozen) == 1:                                            # wenn nur 1 Eintrag in der Zelle ist
#         x = df_meta.loc[df_meta['asin'] == frozen[0]]['title']
#         if x.values.size > 0:                                       # Prüfen ob der Asin in df_meta drin ist
#             all_rules.at[index, 'consequents'] = x

#     elif len(frozen) > 1:                                           # wenn mehr als 1 Eintrag in der Zelle sind
#         names = []
#         for elem in frozen:
#             x = df_meta.loc[df_meta['asin'] == elem]['title']
#             if x.values.size > 0:
#                 names.append(x)
#             else:
#                 names.append(elem)
#             all_rules.at[index, 'consequents'] = names

all_rules = translate_asin(all_rules)

Die gesetzten Filter:
* Support > 0.4% - Paar soll in mindestens 0.4% aller Warenkörbe auftauchen
* Confidence > 60% - wenn A im Korb ist, soll B mit hoher Wahrscheinlichkeit auch drin sein
+ Confidence < 100% - die 'perfekten' Paare sollen nicht beachtet werden

In [68]:
all_rules[(all_rules['support'] > 0.004) & (all_rules['confidence'] > 0.6) & (all_rules['confidence'] < 1)].sort_values(by = 'confidence' , ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,NaN,NaN,0.004820,0.009777,0.004069,0.844262,86.348419,0.004022,6.358271
1,NaN,NaN,0.004938,0.011535,0.004168,0.844000,73.166418,0.004111,6.336312
17,64506 Finish Line Fiber Grip Carbon Fiber B...,455767 Park Tool MLP- 1.2 Chain Plier Name:...,0.005550,0.009145,0.004346,0.782918,85.608633,0.004295,4.564429
0,NaN,NaN,0.006222,0.011535,0.004800,0.771429,66.875196,0.004728,4.324533
15,(B07D7VTQ1R),(B00U8TFZEY),0.006202,0.006419,0.004741,0.764331,119.063988,0.004701,4.216004
6,Hanes Adult EcoSmart Jersey Polo,(B00C6OUDX2),0.005392,0.009521,0.004069,0.754579,79.257383,0.004018,4.035834
5,NaN,NaN,0.005491,0.007921,0.004069,0.741007,93.553544,0.004025,3.830529
16,(B00U8TFZEY),(B07D7VTQ1R),0.006419,0.006202,0.004741,0.738462,119.063988,0.004701,3.799815
14,(B00KQS2NGK),(B003DPVERM),0.007387,0.008671,0.005215,0.705882,81.404797,0.005151,3.370518
10,653675 100 Pcs Alloy Road Mountain Bicycle ...,234383 Shimano Universal Standard Brake Cab...,0.007170,0.012128,0.004938,0.688705,56.786775,0.004851,3.173430


| Antecedents                                                                                                                                                                          | Consequents                                                       | Support | Confidence | Lift   |
|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------|---------|------------|--------|
| Pro Bike Tool CO2 Inflator - Quick &amp; Easy - Presta and Schrader Valve Compatible - Bicycle Tire Pump for Road and Mountain Bikes - Insulated Sleeve - No CO2 Cartridges Included | 16g Threaded CO2 Cartridges 6-Pack Mole-Zap/Ant Zap Refills       | 0.004   | 0.622      | 79.072 |
| 100 Pcs Alloy Road Mountain Bicycle Bike Brake Cable Cap End Tips Crimp Silver                                                                                                       | Shimano Universal Standard Brake Cable Set, For MTB or Road Bikes | 0.004   | 0.689      | 56.787 |
| Finish Line Fiber Grip Carbon Fiber Bicycle Assembly Gel                                                                                                                             | Park Tool MLP- 1.2 Chain Plier                                    | 0.004   | 0.783      | 85.609 |
| Topeak D2 SmartGauge                                                                                                                                                                 | Park Tool MLP- 1.2 Chain Plier                                    | 0.004   | 0.783      | 85.609 |